In [10]:
import pandas as pd
Sales = pd.read_csv('../LTV-Capstone-Project/Data/Online_Sales.csv')
Sales.head(2)

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status
0,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used
1,17850,16680,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used


In [39]:
Sales.columns

Index(['CustomerID', 'Transaction_ID', 'Transaction_Date', 'Product_SKU',
       'Product_Description', 'Product_Category', 'Quantity', 'Avg_Price',
       'Delivery_Charges', 'Coupon_Status'],
      dtype='object')

In [12]:

Marketing_Spend = pd.read_csv('../LTV-Capstone-Project/Data/Marketing_Spend.csv')
Marketing_Spend.head(2)

,Date,Offline_Spend,Online_Spend
0,1/1/2019,4500,2424.50
1,1/2/2019,4500,3480.36


In [33]:
Coupon = pd.read_csv('../LTV-Capstone-Project/Data/Discount_Coupon.csv')
Coupon .head(2)

,Month,Product_Category,Coupon_Code,Discount_pct
0,Jan,Apparel,SALE10,10
1,Feb,Apparel,SALE20,20


In [44]:
Coupon.columns

Index(['Month', 'Product_Category', 'Coupon_Code', 'Discount_pct'], dtype='object')

In [35]:
print(Coupon['Discount_pct'].unique())

[10 20 30]


In [18]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [59]:
CustomersData = pd.read_excel('../LTV-Capstone-Project/Data/CustomersData.xlsx', engine='openpyxl')
CustomersData.head(2)

,CustomerID,Gender,Location,Tenure_Months
0,17850,M,Chicago,12
1,13047,M,California,43


In [36]:
CustomersData.columns

Index(['CustomerID', 'Gender', 'Location', 'Tenure_Months'], dtype='object')

In [29]:
print(CustomersData['Location'].unique())

['Chicago' 'California' 'New York' 'New Jersey' 'Washington DC']


In [41]:
Taxes = pd.read_excel('../LTV-Capstone-Project/Data/Tax_amount.xlsx', engine='openpyxl')
Taxes.head(2)

,Product_Category,GST
0,Nest-USA,0.1
1,Office,0.1


In [43]:
Taxes.columns

Index(['Product_Category', 'GST'], dtype='object')

In [28]:
print(Taxes['Product_Category'].unique())

['Nest-USA' 'Office' 'Apparel' 'Bags' 'Drinkware' 'Lifestyle'
 'Notebooks & Journals' 'Headgear' 'Waze' 'Fun' 'Nest-Canada' 'Backpacks'
 'Google' 'Bottles' 'Gift Cards' 'More Bags' 'Housewares' 'Android'
 'Accessories' 'Nest']


In [45]:
Sales['Month'] = pd.to_datetime(Sales['Transaction_Date']).dt.to_period('M').astype(str)

In [46]:
Sales = Sales.merge(Coupon, on=['Month', 'Product_Category'], how='left')
# Fill missing Discount_pct with 0 if no coupon was applied
Sales['Discount_pct'] = Sales['Discount_pct'].fillna(0)

In [47]:
# 1. Gross Sales
Sales['Gross_Sales'] = Sales['Quantity'] * Sales['Avg_Price']

# 2. Discount Amount
Sales['Discount_Amount'] = Sales['Gross_Sales'] * (Sales['Discount_pct'] / 100)

# 3. Net Subtotal
Sales['Net_Subtotal'] = Sales['Gross_Sales'] - Sales['Discount_Amount']

# 4. Pre-Tax Total (add delivery)
Sales['Pre_Tax_Total'] = Sales['Net_Subtotal'] + Sales['Delivery_Charges']

# 5. Merge GST from Taxes table (if not already done)
Sales = Sales.merge(Taxes, on='Product_Category', how='left')

# 6. GST Amount
Sales['GST_Amount'] = Sales['Pre_Tax_Total'] * (Sales['GST'] / 100)

# 7. Final Invoice (what customer paid)
Sales['Invoice_Value'] = Sales['Pre_Tax_Total'] + Sales['GST_Amount']

# 8. Net Revenue (excludes GST remitted to govt)
Sales['Net_Revenue'] = Sales['Pre_Tax_Total']

In [51]:
Sales.head(2)

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status,...,Coupon_Code,Discount_pct,Gross_Sales,Discount_Amount,Net_Subtotal,Pre_Tax_Total,GST,GST_Amount,Invoice_Value,Net_Revenue
0,17850,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used,...,NaN,0.0,153.71,0.0,153.71,160.21,0.1,0.16021,160.37021,160.21
1,17850,16680,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used,...,NaN,0.0,153.71,0.0,153.71,160.21,0.1,0.16021,160.37021,160.21


In [52]:
historical_ltv = Sales.groupby('CustomerID')['Net_Revenue'].sum().reset_index()
historical_ltv.columns = ['CustomerID', 'Historical_LTV']
print(historical_ltv.head())

   CustomerID  Historical_LTV
0       12346          180.99
1       12347        14500.04
2       12348         1639.27
3       12350         1487.95
4       12356         2079.96


In [53]:
pip install lifetimes

   ---------------------------------------- 0.0/584.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/584.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/584.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/584.2 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/584.2 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/584.2 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/584.2 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/584.2 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/584.2 kB ? eta -:--:--
   -------------------------------------- 584.2/584.2 kB 288.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [54]:
from lifetimes.utils import summary_data_from_transaction_data

# Use the Sales DataFrame with 'CustomerID' and 'Transaction_Date'
summary = summary_data_from_transaction_data(Sales, 
                                             customer_id_col='CustomerID',
                                             datetime_col='Transaction_Date',
                                             monetary_value_col='Net_Revenue',
                                             observation_period_end=Sales['Transaction_Date'].max())

print(summary.head())

            frequency  recency      T  monetary_value
CustomerID                                           
12347             0.0      0.0  169.0            0.00
12348             0.0      0.0   79.0            0.00
12359             0.0      0.0   17.0            0.00
12370             1.0     30.0  106.0          691.79
12377             0.0      0.0   66.0            0.00


In [55]:
from lifetimes import BetaGeoFitter

bgf = BetaGeoFitter()
bgf.fit(summary['frequency'], summary['recency'], summary['T'])

<lifetimes.BetaGeoFitter: fitted with 1153 subjects, a: 0.50, alpha: 61.81, b: 1.46, r: 0.61>

In [56]:
from lifetimes import GammaGammaFitter

# Only use customers with freq > 0
returning_customers = summary[summary['frequency'] > 0]

ggf = GammaGammaFitter()
ggf.fit(returning_customers['frequency'], returning_customers['monetary_value'])

<lifetimes.GammaGammaFitter: fitted with 534 subjects, p: 0.62, q: 3.84, v: 6323.75>

In [57]:
# Predict expected number of purchases
summary['predicted_purchases_6m'] = bgf.conditional_expected_number_of_purchases_up_to_time(180,
            summary['frequency'], summary['recency'], summary['T'])

# Predict expected average revenue
summary['predicted_avg_value'] = ggf.conditional_expected_average_profit(summary['frequency'], summary['monetary_value'])

# Final LTV
summary['predicted_LTV_6m'] = summary['predicted_purchases_6m'] * summary['predicted_avg_value']

# View top customers by predicted LTV
summary[['predicted_LTV_6m']].sort_values(by='predicted_LTV_6m', ascending=False).head()

,predicted_LTV_6m
CustomerID,
12748,40371.721208
15311,23080.506672
17841,18169.249006
14911,15120.130532
14606,13327.959479


In [ ]:
# Merge with demographics or customer segments
ltv_final = summary.reset_index().merge(CustomersData, on='CustomerID', how='left')

# Export to CSV for Tableau
ltv_final.to_csv("Customer_LTV_Summary.csv", index=False)

In [68]:
import os
print(os.getcwd())

c:\Users\dbyst\OneDrive\Desktop\Neue Fische Bootcamp\da-analytics-engineering-project
